In [13]:
# ======================================================================
# SharedAlexNet 로드 + 웹캠 + 그리퍼 서보 제어
# - CPU 사용
# - MAX_FRAMES까지는 계속 돌고, 'q' 누르면 즉시 종료
# - w_n 분포를 고려한 보정 매핑 사용 (GRIP_CLOSE를 70도로 제한)
# ======================================================================

import torch
import torch.nn as nn
from torchvision import models, transforms

import cv2
import math
import numpy as np
import serial, time
from PIL import Image as PILImage

# -----------------------------
# 0. 사용자 환경 설정
# -----------------------------
CKPT_PATH  = r"C:\Users\USER\Downloads\workspace\shared_alexnet_grasp.pth"  # 모델 pth 경로
PORT       = "COM4"     # Arduino IDE -> 도구 -> 포트 에서 확인한 포트 번호
BAUD       = 115200
MAX_FRAMES = 1000       # 이 프레임 수에 도달하면 자동 종료

device = torch.device("cpu")
print("device:", device)

# -----------------------------
# 1. SharedAlexNet 정의
# -----------------------------
class SharedAlexNet(nn.Module):
    def __init__(self, base_model, num_classes, grasp_dim=6):
        super().__init__()
        self.features = base_model.features
        self.avgpool  = base_model.avgpool
        self.shared_fc = nn.Sequential(*list(base_model.classifier[:-1]))
        in_features = base_model.classifier[-1].in_features  # 4096

        self.cls_head = nn.Linear(in_features, num_classes)   # 분류용
        self.reg_head = nn.Linear(in_features, grasp_dim)     # (cx, cy, h, w, sin2θ, cos2θ)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        z = self.shared_fc(x)
        logits = self.cls_head(z)
        grasps = self.reg_head(z)
        return logits, grasps

# -----------------------------
# 2. 체크포인트 로드
# -----------------------------
print("loading checkpoint from:", CKPT_PATH)
ckpt = torch.load(CKPT_PATH, map_location="cpu", weights_only=False)

orig_ids      = ckpt["orig_ids"]
id_to_idx     = ckpt["id_to_idx"]
idx_to_id     = {v: k for k, v in id_to_idx.items()}
imagenet_mean = ckpt["imagenet_mean"]
imagenet_std  = ckpt["imagenet_std"]

num_classes = len(orig_ids)
GRASP_DIM   = 6

print("num_classes:", num_classes)
print("orig_ids:", orig_ids)

try:
    weights = models.AlexNet_Weights.IMAGENET1K_V1
    base_alex = models.alexnet(weights=weights)
except AttributeError:
    base_alex = models.alexnet(pretrained=True)

model = SharedAlexNet(base_alex, num_classes=num_classes, grasp_dim=GRASP_DIM)
model.load_state_dict(ckpt["model_state_dict"])
model.eval()
model.to(device)

print("model loaded.")

# -----------------------------
# 3. 전처리 정의
# -----------------------------
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std),
])

# -----------------------------
# 4. 보정된 w_n -> 그리퍼 각도 매핑
#    - Cornell에서 실제로 나오는 w_n 범위 [W_MIN, W_MAX]를
#      서보 각도 [GRIP_OPEN, GRIP_CLOSE]에 선형 매핑
#    - GRIP_CLOSE를 70도로 제한해서 완전 90도까지 닫히지 않게 함
# -----------------------------
W_MIN = 0.05   # Cornell grasp 폭 하한 근처 (대략값, 나중에 CSV 통계로 조정 가능)
W_MAX = 0.35   # Cornell grasp 폭 상한 근처 (대략값)

GRIP_OPEN  = 5.0   # 완전 벌림 각도 (하드웨어에 맞게 약간 닫힌 상태에서 시작)
GRIP_CLOSE = 70.0  # 최대 닫힘 각도 (90이 아니라 70까지만 닫히게 제한)

def w_to_grip_angle_calibrated(w_n):
    """
    w_n: 네트워크 예측 정규화 grasp 폭 (0~1 근처)

    1) [W_MIN, W_MAX] 로 클램프
    2) 이 구간을 0~1로 다시 정규화
    3) 0~1 를 서보 각도 [GRIP_OPEN, GRIP_CLOSE]에 매핑
       w_n = W_MIN  -> angle = GRIP_CLOSE (가장 닫힘)
       w_n = W_MAX  -> angle = GRIP_OPEN  (가장 벌림)
    """
    w = float(w_n)

    # 1) Cornell 데이터 범위로 클램프
    if w < W_MIN:
        w_clip = W_MIN
    elif w > W_MAX:
        w_clip = W_MAX
    else:
        w_clip = w

    # 2) [W_MIN, W_MAX] -> [0, 1] 정규화
    alpha = (w_clip - W_MIN) / (W_MAX - W_MIN)  # alpha in [0, 1]

    # 3) alpha -> 각도
    angle = GRIP_CLOSE - alpha * (GRIP_CLOSE - GRIP_OPEN)
    return angle

print("test calibrated mapping w_n -> angle:")
for w in [0.0, 0.05, 0.15, 0.25, 0.35, 0.5, 1.0]:
    a = w_to_grip_angle_calibrated(w)
    print(f"  w_n={w:.2f} -> angle={a:.1f}")

# -----------------------------
# 5. 시리얼 오픈
# -----------------------------
print("opening serial:", PORT)
ser = serial.Serial(PORT, BAUD, timeout=1)
time.sleep(2)
print("serial opened.")

# -----------------------------
# 6. 웹캠 오픈
# -----------------------------
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
    ser.close()
    raise SystemExit

print("웹캠 시작. 'Grasp + Gripper' 창에서 'q' 키를 누르면 종료됩니다.")

# -----------------------------
# 7. 메인 루프
# -----------------------------
frame_idx = 0

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("프레임을 읽지 못했습니다.")
            break

        # BGR -> RGB -> PIL
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = PILImage.fromarray(rgb)

        x = preprocess(pil_img).unsqueeze(0).to(device)

        with torch.no_grad():
            logits, grasp = model(x)

        cx_n, cy_n, h_n, w_n, sin2t, cos2t = grasp[0].cpu().numpy()

        theta = 0.5 * math.atan2(sin2t, cos2t)
        theta_deg = theta * 180.0 / math.pi

        # 보정된 매핑 사용
        grip_angle = int(w_to_grip_angle_calibrated(w_n))

        # 아두이노로 전송
        cmd = f"{grip_angle}\n"
        ser.write(cmd.encode("ascii"))

        # 텍스트 오버레이
        text = f"theta={theta_deg:.1f}, w={w_n:.3f}, grip={grip_angle}"
        cv2.putText(frame, text, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.imshow("Grasp + Gripper", frame)

        # 50프레임마다 상태 출력 (디버깅용)
        frame_idx += 1
        if frame_idx % 50 == 0:
            print(f"[frame {frame_idx}] raw w_n={w_n:.4f}, grip_angle={grip_angle}")

        # q 키 누르면 종료
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            print("사용자 q 입력으로 종료.")
            break

        # 프레임 제한에 도달하면 종료 (너무 오래 돌지 않도록)
        if frame_idx >= MAX_FRAMES:
            print(f"MAX_FRAMES({MAX_FRAMES}) 도달, 자동 종료.")
            break

except KeyboardInterrupt:
    print("KeyboardInterrupt로 종료.")

finally:
    cap.release()
    ser.close()
    cv2.destroyAllWindows()
    print("캠/시리얼 종료.")


device: cpu
loading checkpoint from: C:\Users\USER\Downloads\workspace\shared_alexnet_grasp.pth
num_classes: 15
orig_ids: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15)]
model loaded.
test calibrated mapping w_n -> angle:
  w_n=0.00 -> angle=70.0
  w_n=0.05 -> angle=70.0
  w_n=0.15 -> angle=48.3
  w_n=0.25 -> angle=26.7
  w_n=0.35 -> angle=5.0
  w_n=0.50 -> angle=5.0
  w_n=1.00 -> angle=5.0
opening serial: COM4
serial opened.
웹캠 시작. 'Grasp + Gripper' 창에서 'q' 키를 누르면 종료됩니다.
[frame 50] raw w_n=-0.1182, grip_angle=70
[frame 100] raw w_n=0.0307, grip_angle=70
[frame 150] raw w_n=0.1894, grip_angle=39
[frame 200] raw w_n=0.1567, grip_angle=46
[frame 250] raw w_n=0.1396, grip_angle=50
[frame 300] raw w_n=0.1344, grip_angle=51
[frame 350] raw w_n=0.1453, grip_angle=49
[frame 400] raw w_n=0.3233, grip_angle=10
[frame 450] raw w_n=-0.0032, grip